In [ ]:
%pip install datasets argilla gliner -qqq

## Imports

In [1]:
import argilla as rg
from argilla.client.feedback.schemas import SpanValueSchema

from datasets import load_dataset
from gliner import GLiNER

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553 in run_code        │
│                                                                                                  │
│   3550 │   │   │   │   elif async_ :                                                             │
│   3551 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3552 │   │   │   │   else:                                                                     │
│ ❱ 3553 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3554 │   │   │   finally:                                                                      │
│   3555 │   │   │   │   # Reset our crash handler in place                                        │
│   3556 │   │   │   │   sys.excepthook = old_excepthook                                           │
│ in <cell line: 4>:4                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'datasets'

In [2]:
!pip install argilla

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.4/420.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.


## Extract entities

In [ ]:
dataset = load_dataset("HuggingFaceTB/cosmopedia", "stanford", split="train")
sample = dataset.shuffle().select(range(500))

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
model = GLiNER.from_pretrained("urchade/gliner_base")
labels = ["people", "location", "topic", "event"]

entities = model.batch_predict_entities(sample['text'], labels)
entities[0]

## Create Argilla dataset


Setup

In [ ]:
rg.init(
    api_url="<your_argilla_url>",
    api_key="admin.apikey"
)

#rg.FeedbackDataset.from_argilla("cosmopedia ner", workspace="admin").delete()
rg_dataset = rg.FeedbackDataset(
    fields=[rg.TextField(name="text", title="Synthetic text"), rg.TextField(name="prompt", title="Prompt to generate text")],
    questions=[rg.SpanQuestion(name="entities", title="Highlight the entities in the text", field="text", labels=labels), rg.LabelQuestion(name="quality", labels=["👍", "👎"])],
    metadata_properties=[
        rg.TermsMetadataProperty(name="audience"),
        rg.IntegerMetadataProperty(name="text_length")
    ]
)

/usr/local/lib/python3.10/dist-packages/argilla/client/feedback/schemas/questions.py:400: UserWarning: `visible_labels=20` is greater than the total number of labels (4), so it will be set to `4`.
  warnings.warn(msg, UserWarning, stacklevel=1)


Build records and create dataset

In [ ]:
records = [
    rg.FeedbackRecord(
        fields={"text": r['text'], "prompt": r["prompt"]},
        suggestions=[
        {
            "question_name": "entities",
            "value": [
                SpanValueSchema(
                    start=entity['start'],
                    end=entity['end'],
                    label=entity['label']
                )
                for entity in ents
            ],
            "agent": "urchade/gliner_base",
        }
    ],
        metadata={
            "text_length": len(r['text']),
            "audience": r["audience"]
        }
    )
    for r, ents in zip(sample, entities)
]

In [ ]:
rg_dataset.add_records(records)

rg_dataset.push_to_argilla(name="cosmopedia ner", workspace="admin")